In [74]:
import requests
from requests.auth import HTTPBasicAuth
from bs4 import BeautifulSoup

In [75]:
r = requests.get('http://api.elsevier.com/content/search/scopus?query=bee&apiKey=eb73f64c03599119a51db065202c555f')
data = r.json()

In [76]:
login = open("./LoginInfo.txt","r")
user = login.readline()
password = login.readline()
login.close()

In [72]:
abstractPage = requests.get("https://www.scopus.com/inward/record.uri?partnerID=HzOxMe3b&scp=85039924208&origin=inward",auth = HTTPBasicAuth(user,password))
bSoupObj = BeautifulSoup(abstractPage.content,"html5lib")
abstractText = bSoupObj.find("section",{"id":"authorKeywords"}).findAll("li")
print abstractText[1].text

Kalman filter


In [68]:
sKeywords = ""
for word in bSoupObj.find("section",{"id":"indexedKeywords"}).findAll("span"):
    sKeywords+= word.text+", "
print sKeywords[:-2]

Color, Food products, Food storage, Hough transforms, Elliptical shapes, Image moments, Object discrimination, Pollen counting, Pollen discriminations, Receiver operating characteristics, Tracking models, Video monitoring, Kalman filters


In [86]:
f = open("../data/paperTable.tsv","w")
for entry in data['search-results']['entry']:
    #gets abstract text
    abstractPage = requests.get(entry['link'][2]['@href'],auth = HTTPBasicAuth(user,password))
    bSoupObj = BeautifulSoup(abstractPage.content,"html5lib")
    abstractText = bSoupObj.findAll("p")[1].text
    
    #creates lists of keywords, seperated into author keywords and scopus assigned keywords
    aKeywords = ""
    sKeywords = ""
    if bSoupObj.find("section",{"id":"authorKeywords"}) is not None:
        for word in bSoupObj.find("section",{"id":"authorKeywords"}).findAll("li"):
            aKeywords+= word.text+", "
    if bSoupObj.find("section",{"id":"indexedKeywords"}) is not None:
        for word in bSoupObj.find("section",{"id":"indexedKeywords"}).findAll("span"):
            sKeywords+= word.text+", "

    #prints title, abstract, tags, and date created
    f.write(entry['dc:title'].encode("utf8")+"\t"+abstractText.encode("utf8")+"\t"+aKeywords[:-2].encode("utf8")+"\t"+sKeywords[:-2].encode("utf8")+"\t"+entry['prism:coverDate'].encode("utf8")+"\n")
f.close()

### TODO
    -Get Keywords
        >Differentiate between scopus assigned and author assigned
    -Get Year
    -Topic Modeling 
        >ADA
        >LSI
    -Jenkins Automation